### 链接在数据库

In [ ]:
import sys
import json
sys.path.append('../')
from utils import *
read_con = pymysql.connect(host="192.168.126.143",port = 3337,user='root', password='pjzhang', database='ccks_2019',charset='utf8')
cur = read_con.cursor()

In [ ]:
import re
def normalize(sentence):
    sentence = sentence.replace('•','·')
    # 年月日
    items = re.search(r"(\d{4}年\d{2}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
            
    items = re.search(r"(\d{4}年\d{1}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{2}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{1}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
            
    # 月日
    items = re.search(r"(\d{2}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{1}月\d{2}日)",sentence)
    if items:
        for item in items.groups():
            item_ = '0' + item.replace('月','-').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{1}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = '0' + item.replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{2}月\d{1}日)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('月','-0').replace('日','')
            sentence = sentence.replace(item,item_)
    # 年月
    items = re.search(r"(\d{4}年\d{2}月)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-').replace('月','')
            sentence = sentence.replace(item,item_)
    items = re.search(r"(\d{4}年\d{1}月)",sentence)
    if items:
        for item in items.groups():
            item_ = item.replace('年','-0').replace('月','')
            sentence = sentence.replace(item,item_)
    return sentence

sents = ['歌手赵雷在2011年8月7日发行了哪首歌？','哪些人出生在9月19日？','村上春树所写的哪本小说是在2009年5月出版的？']
for sent in sents:
    print(normalize(sent))
    

In [ ]:
# 原始文件
def read_raw(fn):
    # 读取训练文件，返回问句，sql语句，答案的list
    question_list = []
    qindex_list = []
    sql_list = []
    answer_list = []
    with open(fn,'r',encoding='utf-8') as f:
        context=f.read()
    one_data=context.split('\n\n')
    for one in one_data:
        one=one.strip()
        if not one:
            continue
        question,sql,answer=one.split('\n')
        q_index = question[1:question.index(':')]
        question=question[question.index(':')+1:]
        question = normalize(question)
        sql=sql
        answer=[i.strip() for i in answer.strip().split('\t') if i.strip() ]
        question_list.append(question)
        qindex_list.append(q_index)
        sql_list.append(sql)
        answer_list.append(answer)
    return qindex_list,question_list,sql_list,answer_list

qindex_list,question_list,sql_list,answer_list = read_raw("data/test.txt")
question_list

### 读取文件及处理函数

In [ ]:
import re
def sql2ent(sql):
    #patterns=[re.compile(r'{\s?<(.*?)>'),re.compile(r' <(.*?)>\.'),re.compile(r'\.\s?<(.*?)>'),re.compile(r'"(.*)"')]
    entities=[]
    tmp=sql.split('where')
    if len(tmp)==2:
        sql=tmp[1]
    elif len(tmp)==1:
        sql=sql.split('WHERE')[1]
        
    #sql_part=sql.split('.')
    sql_part=re.split(r'(>|[a-z]|")\s?\.',sql)
    #print(sql_part)
    #patterns=[re.compile(r'<.*?>\s<(.*?)>\s?\.'),re.compile(r'\?\w\s<.*?>\s<(.*?)>'),re.compile(r'"(.*?)"')]
    #patterns=[re.compile(r'(<.*?>)\s<.*?>\s\?\s?'),re.compile(r'\?\w\s<.*?>\s(.*?)\s?\.'),re.compile(r"'(.*?)'"),re.compile(r'“(.*?)”')]
    #patterns=[re.compile(r'(<.*?>)\s<.*?>\s\?\s?'),re.compile(r'\?\w\s<.*?>\s(.*?)\s?\.')]
    p1 = re.compile(r'(<.*?>)\s<.*?>\s\?')  # <Y> <R> ?x
    p2 = re.compile(r'\?\w\s<.*?>\s(<.*)') # ?x <R> <Y>.
    p3 = re.compile(r'\?\w\s<.*?>\s(".*)')   # ?x <R> "" 
    p4 = re.compile(r'\?\w\s\?\w\s(<.*)')   # ?x ?x <A>
    p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*)')   # <A> ?x <B>
    p6 = re.compile(r'(<.*?>)\s\?\w\s\?')   # <A> ?x ?x
    p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
    p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")
    patterns=[p1, p2, p3, p4, p5, p6, p7, p8]
    #print(sql_part)
    for sql_item in sql_part:
        for pattern in patterns:
            #tuples=pattern.findall(sql_item+'.')
            tuples=pattern.findall(sql_item)
            #print(sql_item+'.')
            #print('tuples',tuples)
            for es in tuples:
                if not isinstance(es,tuple):
                    entities.append(es)
                    #entities.append(es.split(' ')[-1])
                    continue
                for e in es:
                    entities.append(e)
    entities=list(set(entities))
    out=[]
    for e in entities:
        if not e:
            continue
        elif e[0]=='<' :
            if e[-1]!='>':
                out.append(e+'>')
            else:
                out.append(e)
        elif e[0]=='"' :
            if e[-1]!='"':
                out.append(e+'"')
            else:
                out.append(e)
        elif e[0]=="'":
            if e[-1]!="'":
                out.append(e+"'")
            else:
                out.append(e)
    return out

# def seq2ent2(sql):
#     entities=[]
#     sql=sql.split('where')[1]
#     p1 = re.compile(r'(<.*?>)\s<.*?>\s\?\w')  # <Y> <R> ?x
#     p2 = re.compile(r'\?\w\s<.*?>\s(<.*?>)\.') # ?x <R> <Y>.
#     p3 = re.compile(r'\?\w\s<.*?>\s"(.*?)"\.')   # ?x <R> "" 
#     p4 = re.compile(r'\?\w\s\?\w\s(<.*?>)\.')   # ?x ?x <A>
#     p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*?>)\.')   # <A> ?x <B>
#     p6 = re.compile(r'(<.*?>)\s\?\w\s\?\w\.')   # <A> ?x ?x
#     p7 = re.compile(r'filter regex\(\?\w, "(.*?)"\)')
#     p8 = re.compile(r"filter regex\(\?\w, '(.*?)'\)")
#     patterns=[p1, p2, p3, p4, p5, p6, p7, p8]
#     for pattern in patterns:
#         tuples=pattern.findall(sql)
#         for es in tuples:
#             if not isinstance(es,tuple):
#                 entities.append(es)
#                 continue
#             for e in es:
#                 entities.append(e)
#     entities=list(set(entities))
#     #print(entities)
#     #entities=[e for e in entities if e and (e[0]=='<' or e[0]=='"') ]
#     out=[]
#     for e in entities:
#         if not e:
#             continue
#         elif e[0]=='<' :
#             if e[-1]!='>':
#                 out.append(e+'>')
#             else:
#                 out.append(e)
#         elif e[0]=='"' :
#             if e[-1]!='"':
#                 out.append(e+'"')
#             else:
#                 out.append(e)
#         elif e[0]=="'":
#             if e[-1]!="'":
#                 out.append(e+"'")
#             else:
#                 out.append(e)
#     return out

# sql里实体位置的内容
def sql2ent2(sql):
    entities=[]
    if 'where' in sql:
        sql=sql.split('where')[1]
    else:
        sql=sql.split('WHERE')[1]
    p1 = re.compile(r'{\s{0,2}(<.*?>)\s<.*?>\s\?\w\s?\.')  # <Y> <R> ?x
    p1_ = re.compile(r'\.\s{0,2}(<.*?>)\s<.*?>\s\?\w\s?\.')  # <Y> <R> ?x
    p2 = re.compile(r'\?\w\s<.*?>\s(<.*?>)\s?\.') # ?x <R> <Y>
    p3 = re.compile(r'\?\w\s<.*?>\s(".*?")\s?\.')   # ?x <R> "" 
    p4 = re.compile(r'\?\w\s\?\w\s(<.*?>)\s?\.')   # ?x ?x <A>
    p5 = re.compile(r'(<.*?>)\s\?\w\s(<.*?>)\s?\.')   # <A> ?x <B>
    p6 = re.compile(r'(<.*?>)\s\?\w\s\?\w\s?\.')   # <A> ?x ?x
    p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
    p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")
    p9 = re.compile(r"('.*?')")
    p10 = re.compile(r'(".*?")')
    patterns=[p1, p1_, p2, p3, p4, p5, p6, p7, p8, p9, p10]
    for pattern in patterns:
        tuples=pattern.findall(sql)
        #print(tuples)
        for es in tuples:
            if not isinstance(es,tuple):
                entities.append(es)
                continue
            for e in es:
                entities.append(e)
    entities=list(set(entities))
    #print(entities)
    #entities=[e for e in entities if e and (e[0]=='<' or e[0]=='"') ]
    out=[]
    for e in entities:
        if not e:
            continue
        elif e[0]=='<' :
            if e[-1]!='>':
                out.append(e+'>')
            else:
                out.append(e)
        elif e[0]=='"' :
            if e[-1]!='"':
                out.append(e+'"')
            else:
                out.append(e)
        elif e[0]=="'":
            if e[-1]!="'":
                out.append(e+"'")
            else:
                out.append(e)
        else:
            out.append(e)
    return out

# sql 里的有意义字符
def sql2seq(sql):
    tmp=sql.split('where')
    if len(tmp)==2:
        sql=tmp[1]
    elif len(tmp)==1:
        sql=sql.split('WHERE')[1]
    
    pattern=re.compile(r'(<.*?>|".*?"|\'.*?\')')
    items=pattern.findall(sql)
    return items
    
def entity_data(sql_list):
    entities_list=[]
    for sql in sql_list:
        entities=sql2ent2(sql)
        entities_list.append(entities)
    return entities_list

def rel_data(sql_list):
    tri_list=[]
    for sql in sql_list:
        tri=sql2seq(sql)
        tri_list.append(tri)
    return tri_list

In [ ]:
sql2ent2('select ?x where {  <秦淮八艳_（明末清初秦淮河畔八位名伎）> <本名> ?x . }')

### 处理训练集

### 提取sql语句中有意义的字符串，包括头尾实体及关系，按sql语句中出现的顺序排列

In [ ]:
mode = "valid"# 'train','dev','test'
fn="data/%s.txt"%mode
qindex_list,q_list,s_list,a_list=read_raw(fn)

In [ ]:
# 提取实体
e_list=entity_data(s_list)
not_found=0
not_found_index=[]
for index,i in enumerate(e_list):
    if not i:
        not_found+=1
        not_found_index.append(index)
    print(index,i)
print("not found:",not_found)
print("error question index:")
print(not_found_index)

In [ ]:
# 提取关系
rel_list=rel_data(s_list)
not_found=0
not_found_index=[]
for index,i in enumerate(rel_list):
    if not i:
        not_found+=1
        not_found_index.append(index)
    print(index,i)
print("not found:",not_found)

In [ ]:
# 提取mention
def load_mention(cur,q_list,e_list):
    mention_list=[]
    for index,es in enumerate(e_list):
        mention=[]
        if not es:
            mention_list.append([])
            continue
        for item in es:
            if item[0]=='"' or item[0]=="'":
                if item[1:-1] in q_list[index]:
                    mention.append(item[1:-1])
            else:
                cand=search_alias(item[1:-1],cur)
                cand.append(del_des(item)[1:-1])
                cand.append(item[1:-1])
                tmp=[c for c in cand if c in q_list[index]]
                tmp=sorted(tmp,key=lambda x:len(x),reverse=True)
#             from ipdb import set_trace
#             set_trace()
                if not tmp:
                    mention.append([])
                else:
                    mention.append(tmp[0])
        mention_list.append(mention)
    return mention_list

mention_list=load_mention(cur,q_list,e_list)

In [ ]:
num_unuse=0
for i, items in enumerate(mention_list):
    if not items or [] in items:
        num_unuse+=1
        print(q_list[i],s_list[i],e_list[i])
        print('-'*30)
num_unuse

In [ ]:
# patternHead=re.compile(r'(\?[a-z]) <.*?> "|<')
# patternHead=re.compile(r'\?[a-z] <')
# patternHead.findall('{ <青城派_（金庸小说中的门派）> ?x <掌门人>  . ?x <号称> "三峡以西剑法第一" . }')
del_des('<舒杰_(维力医疗高管)>')

### 问句根据sparql进行分类

In [ ]:
import re
fn_out="data/classes_questions/%s"%mode+"_问句类型%s.txt"
patternVar=re.compile(r'(\?[a-z])')
patternHead=re.compile(r'(\?[a-z]) <')
patternRel=re.compile(r'> (\?[a-z]) <')
patternTail=re.compile(r'(\?[a-z])(?:\.| \.)')

p7 = re.compile(r'filter regex\(\?\w, (".*?")\)')
p8 = re.compile(r"filter regex\(\?\w, ('.*?')\)")

num_q=0
num_one=0
num_one_head=0 # 尾->头
num_one_rel=0 # 找关系
num_one_tail=0 # 头->尾
num_two=0
num_three=0
num_multi=0
num_two_tail0=0 # 样式一（双尾找一头）
num_two_tail1=0 # 样式二（找中间）
num_two_tail2=0 # 样式三（双头找一尾）
pattern1=0 # 样式四（链式找尾实体）
pattern2=0 # 样式六（三尾找一头）
pattern3=0 # 样式七（三头找一尾）
pattern4=0 # 样式五（尾-头-尾)
pattern_new=0 # 新样式（头尾头）
f_type=''
data=''
with open(fn,'r',encoding='utf-8') as fp:
    for line in fp:
        
        if line[0].lower()=='s':
            line=line.strip()
            if 'where' in line:
                line=line.split('where')
                prefix = line[0]
                line=line[1]
            elif 'WHERE' in line:
                line=line.split('WHERE')
                prefix = line[0]
                line=line[1]
            else:
                print('error')
#             data+=line.strip()
#             data+='\n'
            num_q+=1
            
#             if num_q ==2297:
#                 from pdb import set_trace
#                 set_trace()
            
            l=patternVar.findall(line)
            count=len(l)
            
            if len(patternVar.findall(prefix))>1:
                f_type="0"
            
            elif p7.findall(line) or p8.findall(line) or 'UNION' in line or 'union' in line or 'filter' in line or 'regex' in  line:
                num_multi+=1
                f_type="0"
                
            elif count==1:
                num_one+=1
                num_e = len(patternRel.findall(line))
                if num_e == 1:
                    num_one_rel += 1
                    f_type='3' #"两头找关系"
                    continue
                num_e=len(patternHead.findall(line))
                if num_e==1:
                    num_one_head+=1
                    f_type='2' #"尾找头"
                    continue
                num_e=len(patternTail.findall(line))
                if num_e==1:
                    num_one_tail+=1
                    f_type='1' #"头找尾"
                    continue
            elif count==2:
                num_tail=len(patternTail.findall(line))
                num_two+=1
                if num_tail==0:
                    num_two_tail0+=1
                    f_type='6' #"双尾找一头"
                elif num_tail==1:
                    num_two_tail1+=1
                    f_type='8' #"样式二（找中间）"
                elif num_tail==2:
                    num_two_tail2+=1
                    f_type='9' #"样式三（双头找一尾）"
            elif count==3:
                num_three+=1
                num_tail = len(patternTail.findall(line))
                num_head = len(patternHead.findall(line))
                if len(set(l)) == 1:
                      f_type='10'
                elif num_head==1 and num_tail==2:
                    pt=patternTail.findall(line)
                    ph=patternHead.findall(line)
                    if pt[0]==pt[1]:
                        pattern_new+=1
                        f_type='5' #"新样式（头尾头）"
                    else:
                        pattern1+=1
                        f_type='4' #"样式四（链式找尾实体）"
                elif num_head==3:
                    pattern2+=1
                    f_type='10' #"样式六（三尾找一头）"
                elif num_tail==3:
                    pattern3+=1
                    f_type='10' #"样式七（三头找一尾）"
                if num_head==2 and num_tail==1:
                    pattern4+=1
                    f_type='7' #"样式五（尾-头-尾）"
            else:
                num_multi+=1
                f_type="0"
            
        elif line[0].lower()=="q":
            data+=line.strip()
            pass
        elif not line.strip():
            data+="\t%s\n"%f_type
            with open(fn_out%f_type,'a',encoding='utf-8') as f_out:
                f_out.write(data)
                data=''

### 提取句子类型数据

In [ ]:
# 提取句子类型数据
def load_type(num_case,num_type):
    type_list=[0]*num_case
    for i in range(1,num_type):
        cur_fn = fn_out%i
        if not os.path.exists(cur_fn):
            continue
        with open(cur_fn,'r') as f:
            for line in f:
                line=line.strip()
                if line:
                    index=int(line[1:line.index(":")])-1
                    type_list[index]=i
    return type_list
type_list=load_type(len(mention_list),11)
assert len(q_list)==len(type_list)
type_list

### 保存数据

In [ ]:
# def get_dict(qindex=None,q=None,sql=None,ans=None,ent=None,rel=None,mention=None,qtype=None):
    

In [ ]:
# 保存所有问句集中提取的信息
import json
assert(len(qindex_list)==len(e_list))
assert(len(q_list)==len(e_list))
assert(len(q_list)==len(rel_list))
assert(len(q_list)==len(mention_list))
all_data=(qindex_list,q_list,s_list,a_list,e_list,rel_list,mention_list,type_list)
with open("data/%s.json"%mode,'w')as f:
    json.dump(all_data,f,ensure_ascii=False)
print("output %s successful!" % mode)

### 生成NER训练文件

In [ ]:
import numpy as np
import random
#path = 
modes=["train","valid",'test']
fn_out="NER/data/%s_bert_ner_input.txt"

for mode in modes:
    fn = "%s.json"%mode
    with open(fn,'r') as f:
        cur_data = json.load(f)
        question_label=[]
        for q,ms in zip(cur_data[1],cur_data[6]):
            if not ms or [] in ms:
                continue
            flag=False
            for i in ms:
                if isinstance(i,list):
                    flag=True
            if flag:
                continue

            label=['O']*len(q)
            for m in ms:
                start=q.index(m)
                label[start]='B-'
                if len(m)>1:
                    for it in range(start+1,start+len(m)):
                        label[it]='I-'
            question_label.append((q,label))

        with open(fn_out%mode,'w',encoding='utf-8') as f:
            for q,l in question_label:
                for a,b in zip(q,l):
                    f.write("%s\t%s\n"%(a,b))
                f.write('\n')

### 生成test文件

In [ ]:
mode = 'test' 
fn = "%s.json"%mode
with open(fn,'r') as f:
    cur_data = json.load(f)
    questions_test = cur_data[1]

In [ ]:
fn_out_test = fn_out%mode
with open(fn_out_test,'w',encoding='utf-8') as fp:
    for line in questions_test:
        for word in line:
            fp.write("%s\tO\n"%(word))
        fp.write("\n")

## 测试cell

In [ ]:
import json
mode = 'train' 
fn = "%s.json"%mode
with open(fn,'r') as f:
    cur_data = json.load(f)
    questions_test = cur_data[1]
    mentions_test = cur_data[-2]
    sql = cur_data[2]
for q,m,s in zip(questions_test,mentions_test,sql):
    if not m or [] in m:
        #print(q,'\t',m,'\t',s)
        pass
    else:
        print(q,'\t',m,'\t',s)